# Amazon VPC (Virtual Private Cloud)

<img src="../_assets/aws_service_icons/vpc.svg" width="80" alt="Amazon VPC">

## Goals
- Understand what a **VPC** is (and what it is not).
- Know what VPCs are practically used for.
- See a minimal **AWS SDK** pseudo-code workflow (no execution).


## Prerequisites
- Basic networking: IP/CIDR, subnets, routing, NAT, firewalls.
- Basic AWS: Regions and Availability Zones.

> This notebook includes **pseudo-code only**. It does not run any AWS SDK calls.


## What a VPC is

A **VPC (Virtual Private Cloud)** is a *logically isolated* virtual network inside AWS where you define and control networking for your resources.

At a high level, a VPC lets you choose:
- An IP address range (**CIDR block**) for the network (e.g., `10.0.0.0/16`).
- How that address space is split into **subnets** (typically across Availability Zones).
- How traffic flows using **route tables** and network gateways.
- How traffic is allowed/denied using **Security Groups** (stateful) and **Network ACLs** (stateless).

Common connectivity options include:
- **Internet Gateway (IGW)** for public internet access (usually for *public subnets*).
- **NAT Gateway** for outbound internet access from private subnets.
- **VPC endpoints** (Interface/Gateway) to access AWS services privately.
- **VPN / Direct Connect / Transit Gateway** for hybrid networking and multi-VPC connectivity.


## What VPCs are practically used for

In practice, VPCs are used to:
- **Isolate environments** (prod/staging/dev) and reduce blast radius.
- Build a **public/private subnet** layout (public entry points, private internal services).
- Enforce **network security boundaries** and meet compliance constraints.
- Control **ingress and egress** (who can talk to what, and where traffic can leave).
- Connect AWS to **on-premises networks** or other VPCs.
- Keep service traffic **private** using VPC endpoints (e.g., private access to S3/DynamoDB/ECR).

For ML workloads specifically, you often place training/inference compute (e.g., EC2/EKS/SageMaker) in a VPC to securely reach private data stores and internal services.


## Using VPC with the AWS SDK (pseudo-code)
Below is a minimal, **non-executable** sketch of how you might create a VPC with a basic public subnet setup using an AWS SDK (example uses Python + `boto3`).

Notes:
- Creating AWS networking resources can incur cost (e.g., NAT Gateways); plan cleanup.
- In real projects, prefer **IaC** (Terraform/CloudFormation/CDK) over ad-hoc SDK calls.

```python
# PSEUDO-CODE (do not run)

import boto3

region = "us-east-1"
ec2 = boto3.client("ec2", region_name=region)

# 1) Create a VPC
vpc_id = ec2.create_vpc(CidrBlock="10.0.0.0/16")["Vpc"]["VpcId"]

# Optional (common) settings
# ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport={"Value": True})
# ec2.modify_vpc_attribute(VpcId=vpc_id, EnableDnsHostnames={"Value": True})

# 2) Create a subnet
public_subnet_id = ec2.create_subnet(
    VpcId=vpc_id,
    CidrBlock="10.0.1.0/24",
    AvailabilityZone=f"{region}a",
)["Subnet"]["SubnetId"]

# 3) Attach an Internet Gateway and add a default route (0.0.0.0/0)
igw_id = ec2.create_internet_gateway()["InternetGateway"]["InternetGatewayId"]
ec2.attach_internet_gateway(VpcId=vpc_id, InternetGatewayId=igw_id)

rt_id = ec2.create_route_table(VpcId=vpc_id)["RouteTable"]["RouteTableId"]
ec2.create_route(RouteTableId=rt_id, DestinationCidrBlock="0.0.0.0/0", GatewayId=igw_id)
ec2.associate_route_table(RouteTableId=rt_id, SubnetId=public_subnet_id)

# 4) Create a Security Group for resources in the VPC
sg_id = ec2.create_security_group(
    GroupName="example-sg",
    Description="Example SG for VPC resources",
    VpcId=vpc_id,
)["GroupId"]

# Example: launch compute into the VPC (subnet + SG)
# ec2.run_instances(
#     ImageId="ami-...",
#     InstanceType="t3.micro",
#     SubnetId=public_subnet_id,
#     SecurityGroupIds=[sg_id],
#     MinCount=1,
#     MaxCount=1,
# )

# (Optional) Add VPC endpoints to keep service access private (example: S3 gateway endpoint)
# ec2.create_vpc_endpoint(
#     VpcId=vpc_id,
#     VpcEndpointType="Gateway",
#     ServiceName=f"com.amazonaws.{region}.s3",
#     RouteTableIds=[rt_id],
# )
```


## Pitfalls & quick tips

- Pick a **CIDR** that won’t overlap with on-prem/VPN networks if you plan hybrid connectivity.
- Prefer **private subnets by default**; expose only what must be public.
- Watch costs: **NAT Gateways** and cross-AZ traffic can add up.
- Use **Security Groups** for most allow rules; use NACLs when you need subnet-level stateless filtering.
- Use **multiple AZs** for high availability (subnets + routing per AZ).


## References
- AWS Docs: Amazon VPC
- AWS Docs: VPC endpoints
- AWS SDK (boto3): EC2 client (VPC/subnet/route table APIs)
